In [ ]:
import requests
import logging

logging.basicConfig(filename='quickgo_api.log', level=logging.WARNING)

def get_biol_func(go_term):
    url = f"https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/{go_term}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            aspect = data['results'][0]['aspect']
            if aspect == "molecular_function":
                return data['results'][0]['name']
            else:
                logging.warning(f"GO term {go_term} is not a molecular function")
                return ''
        else:
            logging.warning(f"No results for GO term {go_term}")
            return ''
    else:
        logging.error(f"API request failed with status code {response.status_code} for GO term {go_term}")
        return ''

def update_tsv(tsv_file):
    with open(tsv_file, 'r') as f:
        lines = f.readlines()
    header = lines[0].strip()
    header += '\tBiological Molecular Function\n'
    new_lines = [header]
    for line in lines[1:]:
        rsid, go_term = line.strip().split('\t')
        biol_func = get_biol_func(go_term)
        new_lines.append(f"{rsid}\t{go_term}\t{biol_func}\n")
    with open(tsv_file, 'w') as f:
        f.writelines(new_lines)

update_tsv('filtered_go_data.tsv')
